# Analyzing Restaurants in Ahmedabad

In [0]:
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

In [0]:
CLIENT_ID = '4EDTWLS3HUX5C4BFIANTNTGN1VGNCDJDNPC5HX3KU2MKM3SP' # your Foursquare ID
CLIENT_SECRET = 'CN3MRIKHFLGVLQEJBLYRBEM0JBYQS1PCNEX0K4GAAEFQNZBN' # your Foursquare Secret
VERSION = '20190913' # Foursquare API version
downtor_lat = 23.033863
downtor_long = 72.585022
LIMIT = 50
radius = 15000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                downtor_lat, 
                downtor_long, 
                radius, 
                LIMIT)
results = requests.get(url).json()

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if (len(categories_list) == 0) and (categories_list[0]['id'] != '4d4b7105d754a06374d81259'):
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Manek Chowk Khau Gali,Snack Place,23.023505,72.588539
1,Manek Chowk,Fast Food Restaurant,23.023626,72.588553
2,The Sports Club of Gujarat,Cricket Ground,23.040991,72.564349
3,Crossword,Bookstore,23.032656,72.565404
4,Lucky Tea,Tea Room,23.027829,72.581394


In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [26]:
zomato_df = pd.read_csv('gdrive/My Drive/zomato.csv')
zomato_df.drop(['Longitude', 'Latitude', 'Locality', 'Locality Verbose', 'Restaurant ID', 'Switch to order menu', 'Currency', 'Votes'], axis=1, inplace=True)
zomato_df.head()

,Restaurant Name,Country Code,City,Address,Cuisines,Average Cost for two,Has Table booking,Has Online delivery,Is delivering now,Price range,Aggregate rating,Rating color,Rating text
0,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",Italian,1100,1,0,0,3,4.6,3F7E00,Excellent
1,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",Italian,1100,1,0,0,3,4.6,3F7E00,Excellent
2,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","European, Asian",3100,0,0,0,4,4.7,3F7E00,Excellent
3,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Filipino, American, Italian, Bakery",800,0,0,0,3,4.4,5BA829,Very Good
4,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...",American,700,0,0,0,3,4.4,5BA829,Very Good


In [27]:
abad_rest = zomato_df[zomato_df.City == 'Ahmedabad']
abad_rest.head()

,Restaurant Name,Country Code,City,Address,Cuisines,Average Cost for two,Has Table booking,Has Online delivery,Is delivering now,Price range,Aggregate rating,Rating color,Rating text
2608,Millhouse,1,Ahmedabad,"3rd Floor, Ahmedabad One Mall, Vastrapur, Ahme...","Continental, Italian",1200,0,1,1,3,4.2,5BA829,Very Good
2609,Huber & Holly,1,Ahmedabad,"7 B, Pakwan Cross Road, Boadakdev, Ahmedabad","Cafe, Desserts",800,0,1,0,3,4.8,3F7E00,Excellent
2610,La Pino'z Pizza,1,Ahmedabad,"Shop 10, Circle B, Nyay Marg, Bodakdev, Ahmed...",Pizza,500,0,1,1,2,4.5,3F7E00,Excellent
2611,Rp's Pizzeria,1,Ahmedabad,"1, 2, 3 Prime Plaza, Opposite DLA School, NFD ...","Pizza, Italian",550,0,1,1,2,4.5,3F7E00,Excellent
2612,Rp's Pizzeria,1,Ahmedabad,"1, 2, 3 Prime Plaza, Opposite DLA School, NFD ...","Pizza, Italian",550,0,1,1,2,4.5,3F7E00,Excellent


In [28]:
abad_rest.shape

(88, 13)

In [29]:
nearby_venues.shape

(50, 4)